<a href="https://colab.research.google.com/github/yoyo925104/my-first-web-apps/blob/main/%E7%AC%AC14%E5%9B%9E_%E6%84%9F%E6%83%85%E5%88%86%E6%9E%90%E5%AE%9F%E8%B7%B5_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python で日本語文章の感情分析
## huggingface の bert-base-japanese-sentiment

- [huggingface の bert-japanese-finetuned-sentiment](https://huggingface.co/koheiduck/bert-japanese-finetuned-sentiment)
    - 解説記事: [bert-base-japanese-sentiment モデル作者の BERT HANDSON 資料](https://github.com/ydaigo/BERT_HANDSON/blob/master/BERT_HANDS_ON.ipynb)
    - ネガポジ判定 (ポジティヴとネガティヴの binary classification)

In [ ]:
# 必要なライブラリのインストール
!pip install -q transformers
!pip install ipadic
!pip install fugashi
!pip install xformers
!pip install unidic-lite # 20250107コード定義変更のため追記
# !pip install sentencepiece
# !pip install janome fasttext

In [ ]:
# 必要なライブラリのインポート

from transformers import AutoTokenizer, BertForSequenceClassification, pipeline

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('koheiduck/bert-japanese-finetuned-sentiment')

# Load the model
model = BertForSequenceClassification.from_pretrained('koheiduck/bert-japanese-finetuned-sentiment')

In [ ]:
# シンプルな動作確認
# pipeline("やりたい処理方法", model=利用したいモデル, tokenizer=利用したいトーカナイザー)(判定したい文章)
# labelでは「ポジティブ or ネガティブ」が出力される
# scoreにはlabelで出た極性の度合いが1を最大値として出力される

print(pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)("私は幸福である。"))

[{'label': 'POSITIVE', 'score': 0.9851733446121216}]


In [ ]:
print(pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)("難題にぶつかって頭を抱えている。"))

[{'label': 'NEGATIVE', 'score': 0.9908578991889954}]


In [ ]:
# list_textの中に幾つかの文章を入れ込んで、それらの感情分析を順番に行う

list_text = [
             'この人は、この世の中で、いちばんしあわせな人にちがいありません。',
             '芝居小屋もすばらしいし、お客さんもすばらしい人たちでした。',
             'もし中世の時代だったら、おそらく、火あぶりにされたでしょうよ。',
             'みんなのうるさいことといったら、まるで、ハエがびんの中で、ブンブンいっているようでした。',
             'われわれ人間が、こういうことを考えだすことができるとすれば、われわれは、地の中にうめられるまでに、もっと長生きできてもいいはずだが'
]

In [ ]:
for i in list_text:
  print(i)
  print(pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)(i))
  print("==============================")

この人は、この世の中で、いちばんしあわせな人にちがいありません。
[{'label': 'POSITIVE', 'score': 0.8791817426681519}]
芝居小屋もすばらしいし、お客さんもすばらしい人たちでした。
[{'label': 'POSITIVE', 'score': 0.9705815315246582}]
もし中世の時代だったら、おそらく、火あぶりにされたでしょうよ。
[{'label': 'NEGATIVE', 'score': 0.8034874200820923}]
みんなのうるさいことといったら、まるで、ハエがびんの中で、ブンブンいっているようでした。
[{'label': 'NEGATIVE', 'score': 0.5459651947021484}]
われわれ人間が、こういうことを考えだすことができるとすれば、われわれは、地の中にうめられるまでに、もっと長生きできてもいいはずだが
[{'label': 'NEUTRAL', 'score': 0.5141775608062744}]


In [ ]:
output_rows = []

for i in list_text:
    label = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)(i)[0]["label"]
    score = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)(i)[0]["score"]
    output_rows.append([label, score])

output_rows

[['POSITIVE', 0.8791817426681519],
 ['POSITIVE', 0.9705815315246582],
 ['NEGATIVE', 0.8034874200820923],
 ['NEGATIVE', 0.5459651947021484],
 ['NEUTRAL', 0.5141775608062744]]

In [ ]:
import csv

# Define the path and filename for the CSV file
csv_file = "./output.csv"

# Write the output_rows to the CSV file
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(output_rows)

無機質な単語や短いセンテンスだとどうなるのか？

In [ ]:
list(map(pipeline("sentiment-analysis",model=model, tokenizer=tokenizer), ['最悪だ', '暑い', "楽しい", 'こんにちは', '足']))

[[{'label': 'NEGATIVE', 'score': 0.9927927255630493}],
 [{'label': 'NEGATIVE', 'score': 0.9867493510246277}],
 [{'label': 'POSITIVE', 'score': 0.9866198301315308}],
 [{'label': 'NEUTRAL', 'score': 0.9328478574752808}],
 [{'label': 'NEUTRAL', 'score': 0.6107232570648193}]]

最悪、暑いなどはネガティブに捉えられていて、私たちの感覚と相違はなさそう。

こんにちは、足、などには感情を人間も感じないように、機会も感じない。

## まとめ

日本語文章の感情分析を簡単にできるツールを試してみました。

こういったツールを公開していただけていることに感謝ですね。

真面目に感情分析やって、もっと妥当な結果を出そうとすると、トレーニングデータセットを充実させたり、さらに工夫が必要になりそうです。

## 参考

### 解説、まとめ記事

- [【27個掲載】文章、表情、音声の感情分析に使えるデータセットまとめ | Lionbridge AI](https://lionbridge.ai/ja/datasets/15-free-sentiment-analysis-datasets-for-machine-learning/)
    - リソースや極性辞書へのリンクなど
- [筑波大学 乾先生の感情分析リンクまとめ](http://www.nlp.mibel.cs.tsukuba.ac.jp/~inui/SA/links.html)
- [【自然言語処理】感情分析の進め方＆ハマりやすいポイント - Qiita](https://qiita.com/toshiyuki_tsutsui/items/604f92dbe6e20a18a17e)
- [日本語 Sentiment Analyzer を作ってパッケージ化した話 - Ahogrammer](https://hironsan.hatenablog.com/entry/japanese-sentiment-analyzer)
- [ディープラーニングを使って転職会議の企業クチコミデータを感情分析してみる - Qiita](https://qiita.com/terrierscript/items/54daa5aedde599e2637c)
- [小説「天気の子」を丸ごと一冊、感情分析してみた☔️ - Qiita](https://qiita.com/toshiyuki_tsutsui/items/10f52c30fe1504b83ba1)
- [日本語評価極性辞書を利用したPython用Sentiment Analysisライブラリ oseti を公開しました - Qiita](https://qiita.com/yukinoi/items/46aa016d83bb0e64f598)
- [ML-Askでテキストの感情分析 - Qiita](https://qiita.com/yukinoi/items/ef6fb48b5e3694e9659c)
- [bert-base-japanese-sentiment モデル作者の BERT HANDSON 資料](https://github.com/ydaigo/BERT_HANDSON/blob/master/BERT_HANDS_ON.ipynb)

### 感情分析日本語データセット

- [SNOW D18:日本語感情表現辞書 - 長岡技術科学大学 自然言語処理研究室](http://www.jnlp.org/SNOW/D18)
    - 長岡技術科学大学 自然言語処理研究室
    - 約2,000表現を収録し、各表現に対して我々が独自に定義した48分類の感情を付与